In [4]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions,\
                                           paper_pc_completion_experiment_id_best_epoch
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

In [63]:
incomplete_n_samples = 2048
complete_n_samples = 4096

loss = 'chamfer'

if loss == 'emd':
    exp_counter = '9'
else:
    exp_counter = '10'


class_type = 'all_classes'
exp_counter, best_epoch = paper_pc_completion_experiment_id_best_epoch(class_type, loss)

print best_epoch
    
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
experiment_name = str(exp_counter) + '_all_classes_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss
load_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
load_dir = osp.join(load_dir, experiment_name)
response_file = osp.join(load_dir, 'output', 'epoch_' + str(best_epoch), 'detailed_stats.txt')

 73


In [64]:
from collections import defaultdict
combiner = defaultdict(dict)
with open(response_file, 'r') as fin:
    for i, line in enumerate(fin):
        if i == 0: # first line is comment
            continue
        class_id, model_name, scan_id_mixed, coverage, _ = line.rstrip().split(' ')
        coverage = float(coverage)
        scan_id = scan_id_mixed[:5]
        accuracy = float(scan_id_mixed[5:])
        datum_id = class_id + '.' + model_name + '.' + scan_id
            
        combiner[datum_id]['accuracy'] = accuracy
        combiner[datum_id]['coverage'] = coverage

In [65]:
vscan_data = osp.join(top_data_dir, 'OUT', 'vscan_stats')
vscan_data = [f for f in files_in_subdirs(vscan_data, '.txt')]
for class_file in vscan_data:
    class_id = class_file.split('/')[-2]
    with open(class_file, 'r') as fin:
        for i, line in enumerate(fin):
            if i < 3: # first lines are comment
                continue                        
            tag, _, completeness = line.split()
            completeness = float(completeness)
            datum_id = class_id + '.' + tag            
            combiner[datum_id]['completeness'] = completeness

In [71]:
for v in combiner.itervalues():    
    print v

{'completeness': 0.72583, 'coverage': 0.907471, 'accuracy': 0.946289}
{'completeness': 0.342285, 'coverage': 0.746826, 'accuracy': 0.944824}
{'completeness': 0.55957, 'coverage': 0.687988, 'accuracy': 0.836426}
{'completeness': 0.781494, 'coverage': 0.736572, 'accuracy': 0.779541}
{'completeness': 0.546387, 'coverage': 0.70166, 'accuracy': 0.182129}
{'completeness': 0.523926, 'coverage': 0.482666, 'accuracy': 0.746338}
{'completeness': 0.354492, 'coverage': 0.734863, 'accuracy': 0.884277}
{'completeness': 0.857178, 'coverage': 0.814453, 'accuracy': 0.891602}
{'completeness': 0.79248, 'coverage': 0.864258, 'accuracy': 0.886475}
{'completeness': 0.461914, 'coverage': 0.60498, 'accuracy': 0.847412}
{'completeness': 0.763184, 'coverage': 0.679932, 'accuracy': 0.850586}
{'completeness': 0.691162, 'coverage': 0.844238, 'accuracy': 0.923584}
{'completeness': 0.278076, 'coverage': 0.631592, 'accuracy': 0.466309}
{'completeness': 0.657959, 'coverage': 0.877197, 'accuracy': 0.954834}
{'completen

KeyboardInterrupt: 

In [99]:
def pick_range(ranges, val, sorted_ranges=True):
    if sorted_ranges:
        for i, upper in enumerate(ranges):
            if val <= upper:
                return i
        return len(ranges)    
    else:
        raise NotImplementedError()

[ 0.263428   0.391357   0.51709    0.59082    0.797852   0.8871338  1.       ]


In [105]:
all_completeness = np.zeros(len(combiner))

for i, val in enumerate(combiner.itervalues()):
    all_completeness[i] = val['completeness']

prc = [5, 20, 40, 50, 80, 90, 100]
thresholds = np.percentile(all_completeness, prc)

accuracy_bins = defaultdict(list)
coverage_bins = defaultdict(list)

print thresholds

for i, val in enumerate(combiner.itervalues()):    
    in_bin = pick_range(thresholds, val['completeness'])
    accuracy_bins[in_bin].append(val['accuracy'])
    coverage_bins[in_bin].append(val['coverage'])

[ 0.263428   0.391357   0.51709    0.59082    0.797852   0.8871338  1.       ]


In [116]:
for i in range(len(thresholds)):
    print thresholds[i], np.mean(accuracy_bins[i]), np.mean(coverage_bins[i])

0.263428 0.776381493218 0.617437341554
0.391357 0.828844821222 0.701513741329
0.51709 0.855495701811 0.766902284719
0.59082 0.862930299166 0.784457069775
0.797852 0.888291486417 0.839846345212
0.8871338 0.890761974287 0.876396241636
1.0 0.891687181677 0.910950030641
